<h1> Customer Churn Analysis for FloorMat Grocery store</h1>


<h2>Scenario:</h2>
FloorMat is a grocery shop. Customers usually come to the store once every 2 weeks to purchase groceries. Our team has a marketing budget of $1000 to increase sales during the december closing season.  

We have recieved a customer Dataset from operations of store transactions from Jan'2017 - Dec'2019, We need to find the users that are likely to churn and provide coupons.

In the current example we have used a cut-off date of Oct 16 2019.

The cutoff Date symbolizes 2.5 months before the December closing and the grocery store wants to provide coupons to customer that have not shopped with us in a while.

<h2>Procedure:</h2>

1.Create a datetime field called cut-off and populate as Oct 16 2019,

2.Find customers that have not shopped with us since then.<--criteria for churned customers



In [24]:
import pandas as pd
import datetime as dt



data = pd.read_csv("rfm_xmas19.txt", parse_dates=["trans_date"])
group_by_customer = data.groupby("customer_id")
last_transaction = group_by_customer["trans_date"].max()
#print(dt.now()) 
best_churn = pd.DataFrame(data.groupby('customer_id')['trans_date'].max())
#best_churn.reset_index(inplace = True)
best_churn.info()
cutoff_day = dt.datetime(2019, 10, 16)

best_churn['churned'] = ''
best_churn['churned'] =best_churn['trans_date'].apply(lambda n : 1 if n < cutoff_day else 0)

<class 'pandas.core.frame.DataFrame'>
Index: 6889 entries, FM1112 to FM9000
Data columns (total 1 columns):
trans_date    6889 non-null datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 107.6+ KB


In [25]:
#number of people who have churned given the criteria
churned_percent = (best_churn[best_churn['churned'] == 1].count()/best_churn.count())*100
print('{0} people have not shopped with us for past 3 months'.format(round(churned_percent[1],2)))

46.71 people have not shopped with us for past 3 months


In [29]:
best_churn['amount_spent'] = pd.DataFrame(data.groupby('customer_id')['tran_amount'].sum())
best_churn['nr_of_transactions'] = pd.DataFrame(data.groupby('customer_id')['trans_date'].count())

In [37]:
best_churn.drop(['trans_date'], axis = 1)


Create a Weighted score that takes into account the amount spent by the customer and the number of transactions by the customer.

We will scale the two metrics down using min/max normalization((x-x.min)/(x.max-x.min)). 

Use a weighted sum of the two metrics as follows:
<h2>Score = 0.5 * scaled amount spent +0.5 * scaled number of transaction</h2>

In [45]:
best_churn[["nr_of_transactions", "amount_spent"]].describe().loc[["min", "max"]]
deno = (best_churn['nr_of_transactions'].max()- best_churn['nr_of_transactions'].min())
best_churn['scaled_tran'] = (best_churn['nr_of_transactions'] - best_churn['nr_of_transactions'].min()) /deno

deno = (best_churn['amount_spent'].max() - best_churn['amount_spent'].min())
best_churn['scaled_amount'] = (best_churn['amount_spent'] - best_churn['amount_spent'].min()) /deno

best_churn['score'] = (.5*best_churn['scaled_tran'] + 0.5*best_churn['scaled_amount'])*100

best_churn.sort_values('score' , ascending = False, inplace = True)

<h1>Determine the coupon amount and whom to send</h1> 

The department has a budget of $1000. From past data we found that  
30% discount is enticing enough for customers to come to the store as a guestimate. 

Approach:
1. We check what is the average transaction amount at the store.
2. We calculate the amount and calculate 30% of the amount
3. We divide the number from the budget to get 50 people, we can give discounts to.
4. Now we use the best_churn dataset(sorted by score,descding) and pick the first 50 customers and send them a coupon and wait.

Result:

Average Coupon amount(Estimated): $20

Number of coupon recipients : 50

In [53]:

coupon = round(data['tran_amount'].mean()*.3,2)
nr_of_customers = round(1000/coupon,0)
print('We can provide 30% off coupons to {0} people as part of our marketing campaign'.format(nr_of_customers))

We can provide 30% off coupons to 51.0 people as part of our marketing campaign


In [54]:
#Below is the list of top 50 customers, which will recieve the coupons
top_50_churned = best_churn[best_churn['churned'] == 1].head(50)
top_50_churned.to_csv('best_customers.txt')